In [ ]:
!pip install crewai
!pip install 'crewai[tools]'

In [ ]:
from google.colab import userdata
OpenAPIKey=userdata.get('OpenAPIKey')
SearchAPIKey=userdata.get('GoogleSearchAPI')

In [ ]:
import os
os.environ["SERPER_API_KEY"] = SearchAPIKey # serper.dev API key
os.environ["OPENAI_API_KEY"] = OpenAPIKey

In [ ]:
from crewai import Agent
from crewai_tools import SerperDevTool
search_tool = SerperDevTool()

# Creating a senior researcher agent with memory and verbose mode
researcher = Agent(
  role='Senior Researcher',
  goal='Uncover groundbreaking technologies in {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "Driven by curiosity, you're at the forefront of"
    "innovation, eager to explore and share knowledge that could change"
    "the world."
  ),
  tools=[search_tool],
  allow_delegation=True
)

# Creating a writer agent with custom tools and delegation capability
writer = Agent(
  role='Writer',
  goal='Narrate compelling tech stories about {topic}',
  verbose=True,
  memory=True,
  backstory=(
    "With a flair for simplifying complex topics, you craft"
    "engaging narratives that captivate and educate, bringing new"
    "discoveries to light in an accessible manner."
  ),
  tools=[search_tool],
  allow_delegation=False
)

In [ ]:
from crewai import Task

# Research task
research_task = Task(
  description=(
    "Identify the next big trend in {topic}."
    "Focus on identifying pros and cons and the overall narrative."
    "Your final report should clearly articulate the key points"
    "its market opportunities, and potential risks."
  ),
  expected_output='A comprehensive 3 paragraphs long report on the latest AI trends.',
  tools=[search_tool],
  agent=researcher,
)

# Writing task with language model configuration
write_task = Task(
  description=(
    "Compose an insightful article on {topic}."
    "Focus on the latest trends and how it's impacting the industry."
    "This article should be easy to understand, engaging, and positive."
  ),
  expected_output='A 4 paragraph article on {topic} advancements formatted as markdown.',
  tools=[search_tool],
  agent=writer,
  async_execution=False,
  output_file='new-blog-post.md'  # Example of output customization
)

In [ ]:
from crewai import Crew, Process

# Forming the tech-focused crew with enhanced configurations
crew = Crew(
  agents=[researcher, writer],
  tasks=[research_task, write_task],
  process=Process.sequential  # Optional: Sequential task execution is default
)

In [ ]:
# Starting the task execution process with enhanced feedback
result = crew.kickoff(inputs={'topic': 'AI in healthcare'})
print(result)



> Entering new CrewAgentExecutor chain...
First, I should search the internet to understand the latest trends in AI in healthcare.

Action: 
Search the internet

Action Input: 
{
  "search_query": "latest trends in AI in healthcare"
} 

{'message': 'Unauthorized.', 'statusCode': 403}

Thought: 
I am unable to search the internet due to an authorization error. In this case, I should delegate this task to my coworker, the Writer, providing them with all the necessary context and details to effectively perform the task.

Action: 
Delegate work to co-worker

Action Input: 
{
  "coworker": "Writer",
  "task": "Research and summarize latest trends in AI in healthcare",
  "context": "We need to identify the next big trend in AI in healthcare, focusing on its pros and cons, the overall narrative, market opportunities, and potential risks. Your research should be comprehensive and up-to-date, taking into consideration the latest advancements and breakthroughs in the field of AI in healthcare.